In [1]:
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from tensorflow.keras.layers import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
X = []

In [5]:
path = "/content/drive/My Drive/UNI/DATA/faces_data_2/color/"
dirs = os.scandir(path)

i = 0

for entry in dirs :
  if entry.is_file():
    X.append(img_to_array(load_img('/content/drive/My Drive/UNI/DATA/faces_data_2/color/' + entry.name)))

    i += 1
    if i == 10000:
      break

In [6]:
print(len(X))

10000


In [ ]:
X = np.array(X, dtype=float)

In [ ]:
split = int(0.95*len(X))
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain

In [ ]:
print(Xtrain.shape)

(605, 256, 256, 3)


In [ ]:
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='rmsprop', loss='mse')

In [ ]:
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

batch_size = 10

In [ ]:
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

In [ ]:
tensorboard = TensorBoard(log_dir="/content/drive/My Drive/UNI/AM-PROY-2/Full-version/first_run")
model.fit_generator(image_a_b_gen(batch_size), callbacks=[tensorboard], epochs=10, steps_per_epoch=950)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/10
950/950 [==============================] - 233s 233ms/step - loss: 0.0439
Epoch 2/10
950/950 [==============================] - 221s 233ms/step - loss: 0.0014
Epoch 3/10
950/950 [==============================] - 221s 233ms/step - loss: 0.0014
Epoch 4/10
950/950 [==============================] - 221s 233ms/step - loss: 0.0012
Epoch 5/10
950/950 [==============================] - 223s 235ms/step - loss: 0.0011
Epoch 6/10
950/950 [==============================] - 222s 234ms/step - loss: 0.0010
Epoch 7/10
950/950 [==============================] - 222s 234ms/step - loss: 9.1339e-04
Epoch 8/10
950/950 [==============================] - 222s 234ms/step - loss: 8.3868e-04
Epoch 9/10
950/950 [==============================] - 222s 234ms/step - loss: 7.6614e-04
Epoch 10/10
950/950 [==============================] - 222s 234ms/step - loss: 7.0953e-04


In [ ]:
model_json = model.to_json()
with open("/content/drive/My Drive/UNI/AM-PROY-2/Beta-version/model2.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("/content/drive/My Drive/UNI/AM-PROY-2/Beta-version/model_4.h5")

In [ ]:
Xtest = rgb2lab(1.0/255*X[split:])[:,:,:,0]
Xtest = Xtest.reshape(Xtest.shape+(1,))
Ytest = rgb2lab(1.0/255*X[split:])[:,:,:,1:]
Ytest = Ytest / 128

In [ ]:
print(len(Xtest))
print(len(Ytest))

32
32


In [ ]:
print(model.evaluate(Xtest, Ytest, batch_size=batch_size))
print(model.evaluate(Xtest, Ytest, batch_size = 1))

4/4 [==============================] - 0s 69ms/step - loss: 7.6008e-04
0.0007600774988532066
32/32 [==============================] - 0s 4ms/step - loss: 7.6015e-04
0.0007601482793688774


In [ ]:
color_me = []
for filename in os.listdir('/content/drive/My Drive/UNI/DATA/faces_data_2/gray'):
  color_me.append(img_to_array(load_img('/content/drive/My Drive/UNI/DATA/faces_data_2/gray/'+filename)))

print(len(color_me))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

113


In [ ]:
output = model.predict(color_me)
output = output * 128

4/4 [==============================] - 1s 130ms/step


In [ ]:
for i in range(len(output)):
  cur = np.zeros((256, 256, 3))
  cur[:,:,0] = color_me[i][:,:,0]
  cur[:,:,1:] = output[i]
  imsave("/content/drive/My Drive/UNI/AM-PROY-2/Beta-version/result_faces_2/img_"+str(i)+".png", lab2rgb(cur))